In [2]:
import numpy as np
import pandas as pd
import os, random
from tqdm import tqdm # 진행도 시각화를 위한 라이브러리

seed=42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)

## 데이터 불러오기

In [3]:
train_data = pd.read_csv("nsmc_train.csv", index_col=0)
test_data = pd.read_csv("nsmc_test.csv", index_col=0)
print(train_data.shape)
print(test_data.shape)

(149993, 2)
(49999, 1)


In [ ]:
train_data.head()

In [ ]:
x_train = train_data["review"]
y_train = np.array(train_data["rating"])